In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from scipy.spatial.distance import cdist
from ipywidgets import interact, IntSlider, Button, VBox, Output
import ipywidgets as widgets

# --- Paramètres globaux ---
grid_size = 250
range_a = 100
range_b = 50
angle_deg = 30
idw_anisotropy = 1
idw_power = 2.0

# --- Grille de coordonnées ---
xv, yv = np.meshgrid(np.arange(grid_size), np.arange(grid_size))
coords = np.c_[xv.ravel(), yv.ravel()]

# --- Fonction de noyau FFT-MA ---
def spherical_kernel(grid_shape, range_a, range_b, angle_deg):
    ny, nx = grid_shape
    y, x = np.mgrid[0:ny, 0:nx]
    x -= nx // 2
    y -= ny // 2
    angle = np.deg2rad(angle_deg)
    xr = x * np.cos(angle) + y * np.sin(angle)
    yr = -x * np.sin(angle) + y * np.cos(angle)
    h = np.sqrt((xr / range_a)**2 + (yr / range_b)**2)
    cov = np.zeros_like(h)
    mask = h <= 1
    h_masked = h[mask]
    cov[mask] = 1 - 1.5 * h_masked + 0.5 * h_masked**3
    return np.fft.fft2(np.fft.ifftshift(cov))

# --- Simulation FFT-MA ---
def fft_ma_simulation(shape, range_a, range_b, angle_deg):
    ny, nx = shape
    big_shape = (2 * ny, 2 * nx)
    kernel_fft = spherical_kernel(big_shape, range_a, range_b, angle_deg)
    white_noise = np.random.normal(size=big_shape)
    white_noise_fft = np.fft.fft2(white_noise)
    field_fft = white_noise_fft * np.sqrt(kernel_fft)
    field_big = np.fft.ifft2(field_fft).real
    return field_big[:ny, :nx]

# --- Fonction principale de calcul et d'affichage ---
def run_simulation(n_samples):
    z_field = fft_ma_simulation((grid_size, grid_size), range_a, range_b, angle_deg)

    sample_coords = np.vstack([
        np.array([[0, 0], [0, grid_size-1], [grid_size-1, 0], [grid_size-1, grid_size-1]]),
        coords[np.random.choice(len(coords), n_samples - 4, replace=False)]
    ])
    sample_vals = np.array([z_field[y, x] for x, y in sample_coords.astype(int)])

    # Voronoï
    dists = cdist(coords, sample_coords)
    closest_idx = np.argmin(dists, axis=1)
    z_voronoi = sample_vals[closest_idx].reshape(grid_size, grid_size)

    # Triangles
    tri = Delaunay(sample_coords)
    z_triangle = np.full((grid_size, grid_size), np.nan)
    for i in range(grid_size):
        for j in range(grid_size):
            pt = np.array([j, i])
            simplex = tri.find_simplex([pt])[0]
            if simplex >= 0:
                vertices = tri.simplices[simplex]
                A = sample_coords[vertices]
                Z = sample_vals[vertices]
                A_ = np.c_[A, np.ones(3)]
                coeffs = np.linalg.lstsq(A_, Z, rcond=None)[0]
                z_triangle[i, j] = np.dot([j, i, 1], coeffs)

    # IDW anisotrope
    gx, gy = coords[:, 0], coords[:, 1]
    sx, sy = sample_coords[:, 0], sample_coords[:, 1]
    dx = gx[:, None] - sx[None, :]
    dy = gy[:, None] - sy[None, :]
    d_aniso = np.sqrt(dx**2 + (dy * idw_anisotropy)**2)
    with np.errstate(divide='ignore'):
        w = 1 / d_aniso**idw_power
    w[d_aniso == 0] = 1e10
    w /= w.sum(axis=1, keepdims=True)
    z_idw_flat = np.dot(w, sample_vals)
    z_idw = z_idw_flat.reshape(grid_size, grid_size)

    # Affichage
    fig, axs = plt.subplots(2, 2, figsize=(10, 8))
    vmin, vmax = -3, 3

    im0 = axs[0, 0].imshow(z_field, cmap='viridis', vmin=vmin, vmax=vmax)
    axs[0, 0].scatter(sample_coords[:, 0], sample_coords[:, 1], c='r', s=20)
    axs[0, 0].set_title("Champ de référence (FFT-MA)")
    plt.colorbar(im0, ax=axs[0, 0])

    im1 = axs[0, 1].imshow(z_voronoi, cmap='viridis', vmin=vmin, vmax=vmax)
    axs[0, 1].scatter(sample_coords[:, 0], sample_coords[:, 1], c='r', s=20)
    axs[0, 1].set_title("Estimation par polygones")
    plt.colorbar(im1, ax=axs[0, 1])

    im2 = axs[1, 0].imshow(z_triangle, cmap='viridis', vmin=vmin, vmax=vmax)
    axs[1, 0].triplot(sample_coords[:, 0], sample_coords[:, 1], tri.simplices, color='gray', alpha=0.5)
    axs[1, 0].scatter(sample_coords[:, 0], sample_coords[:, 1], c='r', s=20)
    axs[1, 0].set_title("Estimation par triangles")
    plt.colorbar(im2, ax=axs[1, 0])

    im3 = axs[1, 1].imshow(z_idw, cmap='viridis', vmin=vmin, vmax=vmax)
    axs[1, 1].scatter(sample_coords[:, 0], sample_coords[:, 1], c='r', s=20)
    axs[1, 1].set_title("Estimation par IDW anisotrope")
    plt.colorbar(im3, ax=axs[1, 1])

    for ax in axs.flat:
        ax.set_xlim(0, grid_size - 1)
        ax.set_ylim(0, grid_size - 1)
        ax.set_aspect('equal')

    plt.tight_layout()
    plt.show()

# --- Widgets interactifs ---
slider = IntSlider(value=30, min=10, max=100, step=5, description='N données:')
button = Button(description="Calculer")
output = Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        run_simulation(slider.value)

button.on_click(on_button_clicked)
display(VBox([slider, button, output]))


